<a href="https://colab.research.google.com/github/danfolador/recpadroes/blob/main/RecPad_Exercicio04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EXERCÍCIO 04**

Aluno: Daniel Folador Rossi

Matrícula: 20201mpca0306


---
1. Escolha uma base de classificação e compare os classificadores Logistic Regression e KNN do scikit-learn.

2. Use pelo menos duas formas de avaliação e as repita pelo menos 10 vezes.

3. Calcule a média das repetições de cada avaliação.

5. Suba seu jupyter notebook para o GitHub e disponibilize o link da tarefa (não do repositório).



In [10]:
#======================================================
# INICIALIZACAO
#======================================================
import numpy as np
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [11]:
#======================================================
# BANCO DE DADOS
#======================================================

bd_file = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00429/Cryotherapy.xlsx'
bd = pd.read_excel(bd_file) 
X  = bd.iloc[:, 0:-1]
y = bd.iloc[:, -1]

X = pd.DataFrame.to_numpy(X)
y = pd.DataFrame.to_numpy(y)

N_classes = len(set(y))
classes = set(y)

print("------------")
print(bd.keys())
print("X.shape=",X.shape)
print("y.shape=",y.shape)
print("No de classes=",N_classes)
print("classes=",classes)
print("------------")

------------
Index(['sex', 'age', 'Time', 'Number_of_Warts', 'Type', 'Area',
       'Result_of_Treatment'],
      dtype='object')
X.shape= (90, 6)
y.shape= (90,)
No de classes= 2
classes= {0, 1}
------------


In [12]:
#==============================================================
# RODADAS
#==============================================================

# Padronizacao
pad = StandardScaler()
X_pad = pad.fit_transform(X)

# Modelos
classificadores = [LogisticRegression(),
                    KNeighborsClassifier(n_neighbors=5)]

# Nomes dos classificadores
nomes = ["LR",
          "KNN"]

# Titulo das avaliacoes
title_aval = ["AVALIACAO 1 - PADRONIZACAO + K-FOLD",
              "AVALIACAO 2 - VALIDACAO ANINHADA"]

#--------------------------------------------------------------
# LOOP 1 - Pra cada avaliação
#--------------------------------------------------------------
results_aval1, results_aval2 = [], []
for aval in range(2):
    
    #----------------------------------------------------------
    # LOOP 2 - Pra cada classificador
    #----------------------------------------------------------
    lin = []
    for nome, clf in zip(nomes, classificadores):
        
        #------------------------------------------------------
        # LOOP 3 - As dez rodadas
        #------------------------------------------------------ 
        for rod in range(10):

            # Pipeline
            pip = Pipeline([
                           ("padronizacao", pad),
                           ("classificador", clf)
                           ])
            #--------------------------------------------------
            # AVALIACAO 1 - PADRONIZACAO + K-FOLD
            #-------------------------------------------------- 
            if aval == 0:
                modelo = clf
                X_ = X_pad
                hiperparam = {'n_neighbors': [3,5,7,9,11]}
            
            #--------------------------------------------------
            # AVALIACAO 2 - VALIDACAO ANINHADA
            #--------------------------------------------------
            if aval == 1:
                modelo = pip
                X_ = X
                hiperparam = {'classificador__n_neighbors': [3,5,7,9,11]}


            # GridSearch para validar hiperparametros                
            if nome == "KNN":
                modelo = GridSearchCV(modelo, hiperparam)

            #--------------------------------------------------
            # Ciclos de K-fold
            # (mistura as samples, mas mantém a mesma mistura para cada rodada rod)
            #--------------------------------------------------             
            kf = KFold(n_splits=10, shuffle=True, random_state=(rod))
            ciclos_score =[]
            for train_set, test_set in kf.split(X_):
                modelo.fit(X_[train_set], y[train_set])
                ypred = modelo.predict(X_[test_set])
                score = metrics.accuracy_score(y[test_set], ypred)
                ciclos_score.append(score)
                media_ciclos = np.mean(ciclos_score)
                
            # Resultados
            lin.append([nome, rod, ciclos_score, media_ciclos])   
    
    if aval == 0: results_aval1.extend(lin)
    if aval == 1: results_aval2.extend(lin)

In [13]:
#==============================================================
# RESULTADOS
#==============================================================
for titles, results in zip(title_aval, [results_aval1, results_aval2]):

    results = np.array(results)

    #----------------------------------------------------------
    # Obtem a tabela resultado, usando o classificador como índice de linha
    #---------------------------------------------------------- 
    df = pd.DataFrame(data=results[:,1:],
                      index=results[:,0],
                      columns=["Rodada", "Folds Score (accuracy)", "Media"]
                      )
    
    #----------------------------------------------------------
    # Obtem a media das avalicoes por classificador
    #---------------------------------------------------------- 
    df_LR = df.filter(like='LR', axis=0)
    df_KNN = df.filter(like='KNN', axis=0)
    med_LR = np.mean(df_LR.loc[:,"Media"])
    med_KNN = np.mean(df_KNN.loc[:,"Media"])

    
    print("=================================================================")
    print(titles)
    print("=================================================================")
    print(df)
    print("-----------------------")
    print("Media para LR=" , med_LR)
    print("Media para KNN=" , med_KNN)
    print("")

AVALIACAO 1 - PADRONIZACAO + K-FOLD
    Rodada                             Folds Score (accuracy)     Media
LR       0  [0.8888888888888888, 1.0, 1.0, 0.8888888888888...  0.855556
LR       1  [0.8888888888888888, 1.0, 1.0, 0.8888888888888...  0.888889
LR       2  [0.6666666666666666, 1.0, 0.6666666666666666, ...  0.855556
LR       3  [0.8888888888888888, 1.0, 0.4444444444444444, ...  0.833333
LR       4  [0.8888888888888888, 0.7777777777777778, 0.888...  0.877778
LR       5  [1.0, 0.8888888888888888, 0.7777777777777778, ...  0.866667
LR       6  [1.0, 0.8888888888888888, 0.7777777777777778, ...  0.877778
LR       7  [1.0, 0.7777777777777778, 0.7777777777777778, ...  0.855556
LR       8  [0.8888888888888888, 0.8888888888888888, 0.777...  0.888889
LR       9  [1.0, 0.8888888888888888, 0.7777777777777778, ...  0.877778
KNN      0  [1.0, 1.0, 0.6666666666666666, 0.8888888888888...  0.877778
KNN      1  [0.8888888888888888, 0.7777777777777778, 1.0, ...  0.877778
KNN      2  [0.6666666666666